In [1]:
import docplex
import pandas as pd
import tensorflow as tf
import numpy as np
import utility
import copy
import mlp_explainer
import mymetrics
import time
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn import datasets
from sklearn.model_selection import train_test_split
from milp import codify_network
from teste import get_minimal_explanation
from sklearn.metrics import classification_report
import pickle

In [2]:
#SklearnDatasets
dataset = datasets.load_breast_cancer()
df = pd.DataFrame(dataset.data, columns = dataset.feature_names)
scaler = MinMaxScaler()
scaler.fit(dataset.data)
scaled_df = scaler.transform(dataset.data)
lower_bound = scaled_df.min()
upper_bound = scaled_df.max()
print(lower_bound, upper_bound)
df_scaled = pd.DataFrame(scaled_df, columns=df.columns)
targets = (utility.check_targets_0_1(np.where(dataset.target == dataset.target[0],0,1))).astype(np.int32)
df_scaled['target'] = targets
columns = df_scaled.columns
dataset_name = 'Breast_Cancer'
result_path = f'{dataset_name}_results'
if not os.path.exists(result_path):
    os.makedirs(result_path)
    print(f"Created directory: {result_path}")
else:
    print(f"Directory already exists: {result_path}")
display(df_scaled)

0.0 1.0000000000000002
Original Targets:  [0 1] 
Desired Targets: [0,1]
Is original the desired [0, 1]?  True
Directory already exists: Breast_Cancer_results


,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,0.521037,0.022658,0.545989,0.363733,0.593753,0.792037,0.703140,0.731113,0.686364,0.605518,...,0.141525,0.668310,0.450698,0.601136,0.619292,0.568610,0.912027,0.598462,0.418864,0
1,0.643144,0.272574,0.615783,0.501591,0.289880,0.181768,0.203608,0.348757,0.379798,0.141323,...,0.303571,0.539818,0.435214,0.347553,0.154563,0.192971,0.639175,0.233590,0.222878,0
2,0.601496,0.390260,0.595743,0.449417,0.514309,0.431017,0.462512,0.635686,0.509596,0.211247,...,0.360075,0.508442,0.374508,0.483590,0.385375,0.359744,0.835052,0.403706,0.213433,0
3,0.210090,0.360839,0.233501,0.102906,0.811321,0.811361,0.565604,0.522863,0.776263,1.000000,...,0.385928,0.241347,0.094008,0.915472,0.814012,0.548642,0.884880,1.000000,0.773711,0
4,0.629893,0.156578,0.630986,0.489290,0.430351,0.347893,0.463918,0.518390,0.378283,0.186816,...,0.123934,0.506948,0.341575,0.437364,0.172415,0.319489,0.558419,0.157500,0.142595,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,0.690000,0.428813,0.678668,0.566490,0.526948,0.296055,0.571462,0.690358,0.336364,0.132056,...,0.383262,0.576174,0.452664,0.461137,0.178527,0.328035,0.761512,0.097575,0.105667,0
565,0.622320,0.626987,0.604036,0.474019,0.407782,0.257714,0.337395,0.486630,0.349495,0.113100,...,0.699094,0.520892,0.379915,0.300007,0.159997,0.256789,0.559450,0.198502,0.074315,0
566,0.455251,0.621238,0.445788,0.303118,0.288165,0.254340,0.216753,0.263519,0.267677,0.137321,...,0.589019,0.379949,0.230731,0.282177,0.273705,0.271805,0.487285,0.128721,0.151909,0
567,0.644564,0.663510,0.665538,0.475716,0.588336,0.790197,0.823336,0.755467,0.675253,0.425442,...,0.730277,0.668310,0.402035,0.619626,0.815758,0.749760,0.910653,0.497142,0.452315,0


In [3]:
np.random.seed(50)
X_train, X_test, y_train, y_test = train_test_split(scaled_df, targets, test_size=0.75,random_state=50,stratify=targets)
X = np.concatenate((X_train,X_test),axis=0)
y = np.concatenate((y_train,y_test),axis=0)

training_data = pd.DataFrame(X_train, columns = columns[:-1])
training_data[columns[-1]] = y_train
testing_data = pd.DataFrame(X_test, columns = columns[:-1])
testing_data[columns[-1]] = y_test
dataframe = pd.concat([training_data, testing_data])
data = dataframe.to_numpy()
n_classes = dataframe['target'].nunique()

original_bounds = [[dataframe[dataframe.columns[i]].min(),dataframe[dataframe.columns[i]].max()] for i in range(len(dataframe.columns[:-1]))]
keras_model = tf.keras.models.load_model(f'new_models/{dataset_name}.h5')

In [4]:
mp_model, output_bounds = codify_network(keras_model, dataframe, 'fischetti', relax_constraints=False)
with open(f'{result_path}/{dataset_name}_mp_model.pkl', 'wb') as f:
    pickle.dump(mp_model, f)

with open(f'bounds/{dataset_name}_output_bounds.pkl', 'wb') as f:
    pickle.dump(output_bounds, f)

In [5]:
predictions = []
possible_classes = np.unique(y_test)
class_indexes = []
class_predictions = []
for i in range(n_classes):
    class_indexes.append([])
    class_predictions.append([])
possible_classes, class_indexes, class_predictions
data = testing_data.to_numpy()
for i in range(len(data)):
    predictions.append(mlp_explainer.model_classification_output(k_model=keras_model, net_input=data[i, :-1])[1].numpy())    
    for j,p_class in enumerate(possible_classes):
        if predictions[-1] == p_class:
            class_indexes[j].append(i)
            class_predictions[j].append(data[i, :-1])
print("Metrics:", classification_report(testing_data.to_numpy()[:, -1], predictions,digits=4))

Metrics:               precision    recall  f1-score   support

         0.0     0.9934    0.9434    0.9677       159
         1.0     0.9674    0.9963    0.9816       268

    accuracy                         0.9766       427
   macro avg     0.9804    0.9698    0.9747       427
weighted avg     0.9771    0.9766    0.9765       427



In [6]:
cols = list(testing_data.columns)
if 'target' not in cols:
    cols.append('target')
predicted_dataset = []
for i,pos_class in enumerate(np.unique(y_test)):
    for instance in (testing_data.to_numpy()[:, :-1][class_indexes[i]]):
        instance = np.append(instance, pos_class.astype('int'))
        predicted_dataset.append(instance)
predicted_dataset = np.asarray(predicted_dataset)
pred_dataset_df = pd.DataFrame(predicted_dataset, columns=cols)
pred_dataset_df['target'] = pred_dataset_df['target'].astype('int')

In [7]:
# Save Bounds to CSV
np.savez(f'bounds/{dataset_name}_data_bounds.npz', original_bounds=original_bounds)
# Save Testing Set to CSV
pred_dataset_df.to_csv(f'{dataset_name}_results/{dataset_name}_X_test.csv', index=False)

In [7]:
metrics_dataframes = []
times_onestep = []
sizes_onestep = []
rsum_onestep = []
coverage_onestep = []
pos_exp_onestep = []
neg_exp_onestep = []

In [8]:
original_bounds

[[0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 0.9999999999999999],
 [0.0, 1.0],
 [0.0, 0.9999999999999999],
 [0.0, 0.9999999999999999],
 [0.0, 0.9999999999999999],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 0.9999999999999999],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0000000000000002],
 [0.0, 1.0],
 [0.0, 0.9999999999999999],
 [0.0, 1.0],
 [0.0, 0.9999999999999999],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 0.9999999999999999],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 0.9999999999999998],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 1.0],
 [0.0, 0.9999999999999999],
 [0.0, 1.0]]

In [9]:
onestep_explanations = []
for j in range(len(pred_dataset_df['target'].unique())):
    for i, sample in enumerate((testing_data.to_numpy()[:, :-1][class_indexes[j]])):
        start = time.perf_counter()
        explanation, minimal = mlp_explainer.run_explanation(sample = sample, n_classes=n_classes, kmodel=keras_model, model=mp_model, output_bounds=output_bounds, og_bounds=original_bounds, enable_log=False,
                                                             )
        end = time.perf_counter()
        onestep_explanations.append(explanation)
        times_onestep.append(end-start)
        sizes_onestep.append(len(minimal))
        rsum_onestep.append(mymetrics.range_sum(onestep_explanations[-1]))
        coverage_onestep.append(len(mymetrics.calculate_coverage(testing_data, onestep_explanations[-1])))

In [10]:
def compute_mean_std(arr):
    return np.mean(arr), np.std(arr)

def relative_percentage_diff(new, old):
    if np.any(old == 0):
        print(f'Warning: found possible division by zero')
        return np.where(old != 0, ((new - old) / old) * 100, np.nan)
    return ((new - old) / old) * 100

p_values = [0.75] #[0.25, 0.50, 0.75]
for p_value in p_values:
    print(f"p = {p_value}")
    times_twostep = []
    sizes_twostep = []
    rsum_twostep = []
    coverage_twostep = []
    twostep_explanations = []
    for j in range(len(pred_dataset_df['target'].unique())):
        for i, sample in enumerate((testing_data.to_numpy()[:, :-1][class_indexes[j]])):
            start = time.perf_counter()
            
            explanation, minimal = mlp_explainer.run_explanation_doublestep(sample = sample, n_classes=n_classes, kmodel=keras_model, model=mp_model, output_bounds=output_bounds, og_bounds=original_bounds, p=p_value)
            end = time.perf_counter()
            twostep_explanations.append(explanation)
            times_twostep.append(end-start)
            sizes_twostep.append(len(minimal))
            rsum_twostep.append(mymetrics.range_sum(twostep_explanations[-1]))
            
            coverage_twostep.append(len(mymetrics.calculate_coverage(testing_data, twostep_explanations[-1])))

    times_onestep = np.array(times_onestep)
    times_twostep = np.array(times_twostep)
    sizes_onestep = np.array(sizes_onestep)
    sizes_twostep = np.array(sizes_twostep)
    rsum_onestep = np.array(rsum_onestep)
    rsum_twostep = np.array(rsum_twostep)
    coverage_onestep = np.array(coverage_onestep)
    coverage_twostep = np.array(coverage_twostep)

    # Compute means and standard deviations
    (time_mean_onestep, time_std_onestep) = compute_mean_std(times_onestep)
    (time_mean_twostep, time_std_twostep) = compute_mean_std(times_twostep)
    
    (sizes_mean_onestep, sizes_std_onestep) = compute_mean_std(sizes_onestep)
    (sizes_mean_twostep, sizes_std_twostep) = compute_mean_std(sizes_twostep)
    
    (rsum_mean_onestep, rsum_std_onestep) = compute_mean_std(rsum_onestep)
    (rsum_mean_twostep, rsum_std_twostep) = compute_mean_std(rsum_twostep)
    
    (coverage_mean_onestep, coverage_std_onestep) = compute_mean_std(coverage_onestep)
    (coverage_mean_twostep, coverage_std_twostep) = compute_mean_std(coverage_twostep)

    # Compute relative percentage differences (Mean & Std)
    time_mean_diff = relative_percentage_diff(time_mean_twostep, time_mean_onestep)
    sizes_mean_diff = relative_percentage_diff(sizes_mean_twostep, sizes_mean_onestep)
    rsum_mean_diff = relative_percentage_diff(rsum_mean_twostep, rsum_mean_onestep)
    coverage_mean_diff = relative_percentage_diff(coverage_mean_twostep, coverage_mean_onestep)
    
    time_std_diff = relative_percentage_diff(time_std_twostep, time_std_onestep)
    sizes_std_diff = relative_percentage_diff(sizes_std_twostep, sizes_std_onestep)
    rsum_std_diff = relative_percentage_diff(rsum_std_twostep, rsum_std_onestep)
    coverage_std_diff = relative_percentage_diff(coverage_std_twostep, coverage_std_onestep)
    
    # Compute pointwise relative differences
    time_relative_pointwise = relative_percentage_diff(times_twostep, times_onestep)
    sizes_relative_pointwise = relative_percentage_diff(sizes_twostep, sizes_onestep)
    rsum_relative_pointwise = relative_percentage_diff(rsum_twostep, rsum_onestep)
    coverage_relative_pointwise = relative_percentage_diff(coverage_twostep, coverage_onestep)
    
    # Compute pointwise means
    time_relative_mean = np.mean(time_relative_pointwise) 
    sizes_relative_mean = np.mean(sizes_relative_pointwise)
    rsum_relative_mean = np.mean(rsum_relative_pointwise)
    coverage_relative_mean = np.mean(coverage_relative_pointwise)
    
    # Compute pointwise standard deviations
    time_relative_std = np.std(time_relative_pointwise) 
    sizes_relative_std = np.std(sizes_relative_pointwise)
    rsum_relative_std = np.std(rsum_relative_pointwise)
    coverage_relative_std = np.std(coverage_relative_pointwise)
    
    # Organize Data
    all_metrics_data = {
        'Metric': ['Time', 'Size', 'Ranges_Sum', 'Coverage'],
        'ONESTEP_MEAN': [time_mean_onestep, sizes_mean_onestep, rsum_mean_onestep, coverage_mean_onestep],
        'ONESTEP_STD': [time_std_onestep, sizes_std_onestep, rsum_std_onestep, coverage_std_onestep],
        'TWOSTEP_MEAN': [time_mean_twostep, sizes_mean_twostep, rsum_mean_twostep, coverage_mean_twostep],
        'TWOSTEP_STD': [time_std_twostep, sizes_std_twostep, rsum_std_twostep, coverage_std_twostep],
        'MEAN_DIFF_%': [time_mean_diff, sizes_mean_diff, rsum_mean_diff, coverage_mean_diff],
        'STD_DIFF_%': [time_std_diff, sizes_std_diff, rsum_std_diff, coverage_std_diff],
        'POINTWISE_MEAN_%': [time_relative_mean, sizes_relative_mean, rsum_relative_mean, coverage_relative_mean],
        'POINTWISE_STD_%': [time_relative_std, sizes_relative_std, rsum_relative_std, coverage_relative_std]
    }
    # Display and save
    all_metrics_df = pd.DataFrame(all_metrics_data)
    display(all_metrics_df)
    all_metrics_df.to_csv(f'{result_path}/results_{p_value}.csv', index=False)
    
    #Save Raw Metric Data
    raw_df = pd.DataFrame({
        "times_onestep": times_onestep, 
        "times_twostep": times_twostep,
        "sizes_onestep": sizes_onestep, 
        "sizes_twostep": sizes_twostep,
        "rsum_onestep": rsum_onestep, 
        "rsum_twostep": rsum_twostep,
        "coverage_onestep": coverage_onestep, 
        "coverage_twostep": coverage_twostep,
        "time_relative_%": time_relative_pointwise,
        "sizes_relative_%": sizes_relative_pointwise,
        "rsum_relative_%": rsum_relative_pointwise,
        "coverage_relative_%": coverage_relative_pointwise
    })
    
    # Save to CSV
    raw_df.to_csv(f"{result_path}/raw_metric_data_{p_value}.csv", index=False)

    # Save onestep explanations
    np.savez(f'{result_path}/onestep_explanations_{p_value}.npz', 
             onestep_explanations=onestep_explanations)
    
    # Save twostep explanations
    np.savez(f'{result_path}/twostep_explanations{p_value}.npz', 
             twostep_explanations=twostep_explanations)

p = 0.75


,Metric,ONESTEP_MEAN,ONESTEP_STD,TWOSTEP_MEAN,TWOSTEP_STD,MEAN_DIFF_%,STD_DIFF_%,POINTWISE_MEAN_%,POINTWISE_STD_%
0,Time,1.903408,0.489949,3.421303,0.731675,79.746150,49.337081,87.199198,47.809679
1,Size,20.419204,3.566682,20.419204,3.566682,0.000000,0.000000,0.000000,0.000000
2,Ranges_Sum,16.670096,2.760853,17.074700,2.000323,2.427124,-27.546947,5.336533,25.136787
3,Coverage,1.063232,0.318415,1.067916,0.331876,0.440529,4.227425,0.351288,5.399130
